In [47]:
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchaudio
import torchaudio.models as models
import torchvision.models as vmodels
import torchaudio.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import glob
import librosa
import librosa.display
import soundfile as sf
from scipy.interpolate import interp1d
import IPython.display as ipd
import ast
import time
import sys

In [2]:
# Hyper parameters
EPOCHS = 10
TRAIN_BATCH_SIZE = 4
N_MELS = 256
MAXLEN = 800

In [3]:
class MelSpecDataset(Dataset):
    def __init__(self, source_file):
        self.source = pd.read_csv(source_file)
        
        mel_specs = []
        wvs = []
        for idx in range(len(self.source)):
            ex = self.source.iloc[idx]
            waveform = ex.waveform

            if isinstance(waveform, str): 
                wv = ','.join(ex.waveform.replace('[ ', '[').split())
                wv = np.array(ast.literal_eval(wv))
                waveform = torch.from_numpy(wv).view(1, -1).to(dtype=torch.float32)

                wvs.append(waveform)

            sample_rate = int(ex.sample_rate)
            
            waveform = waveform.view(1, 1, -1)
            
            mel_spec = transforms.MelSpectrogram(sample_rate=sample_rate, n_mels=N_MELS)(waveform)
            mel_spec = self.__reshape(mel_spec[0])
                        
            #mfcc = librosa.feature.mfcc(y=wv, sr=ex.sample_rate)

            mel_specs.append(mel_spec)

        if 'mel_spec' in self.source:
            self.source = self.source.assign(mel_spec=mel_specs)
        else:
            self.source.insert(4, "mel_spec", mel_specs, True)
            
        # drop waveform data for now to save memory
        self.source = self.source.drop(columns=['waveform'])
        
        # uncomment to visualize waveform
        #if len(wvs) > 0:
        #    self.source = self.source.assign(waveform=wvs)
    def __reshape(self, mel_spec):
        diff = MAXLEN - mel_spec.shape[2]
        if diff > 0:
            # pad
            s = int(diff/2) 
            e = diff - s
            mel_spec = F.pad(input=mel_spec, pad=(s, e, 0, 0, 0, 0), mode='constant', value=0)
        else:
            # trim
            mel_spec = mel_spec.narrow(2, diff*-1, MAXLEN)

        mel_spec = mel_spec.repeat(3, 1, 1)
        return mel_spec
    def get_waveform(self, idx):
        ex = self.source.iloc[idx]
        return (ex.waveform, ex.sample_rate)
    def __getitem__(self, idx):
        ex = self.source.iloc[idx]
        return (ex.mel_spec, int(ex.species_id))
    def __len__(self):
        return len(self.source)

In [4]:
def reshape_mel_spec(mel_spec):
        diff = MAXLEN - mel_spec.shape[2]
        if diff > 0:
            # pad
            s = int(diff/2) 
            e = diff - s
            mel_spec = F.pad(input=mel_spec, pad=(s, e, 0, 0, 0, 0), mode='constant', value=0)
        else:
            # trim
            mel_spec = mel_spec.narrow(2, diff*-1, MAXLEN)

        mel_spec = mel_spec.repeat(3, 1, 1)
        return mel_spec

In [5]:
class RainforestNet(nn.Module):
    def __init__(self):
        super(RainforestNet, self).__init__()
        self.resnet = vmodels.resnet50(pretrained=True)
        self.fc1 = nn.Linear(1000, 500)
        self.fc2 = nn.Linear(500, 225)
        self.fc3 = nn.Linear(225, 24)

    def forward(self, x):
        x = self.resnet(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = self.fc2(x)
        x = self.fc3(x)

        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [5]:
trainset = MelSpecDataset('train_classified-80-percent.csv')
trainloader = torch.utils.data.DataLoader(trainset, batch_size=TRAIN_BATCH_SIZE,shuffle=True)

In [12]:
# CUDA
is_cuda = True and torch.cuda.is_available()
device = torch.device('cuda') if is_cuda else torch.device('cpu')

print ('cuda: ', is_cuda)

torch.cuda.empty_cache()

cuda:  True


In [7]:
#resnet = vmodels.resnet50(pretrained=True).to(device)
model = RainforestNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [8]:
print (len(trainset))

1048


In [9]:
# when tensors are of the same shape from the dataset, will use trainloader instead of trainset directly

start_time = time.time()

for epoch in range(EPOCHS):
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        #print (inputs.shape)
        #print (labels)
                        
        optimizer.zero_grad()
                
        output = model(inputs)
        loss = criterion(output, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d %5d] loss: %.3f' %
                    (epoch+1, i+1, running_loss / 50))
            running_loss = 0.0
            
end_time = time.time()
print ('\n-------- DONE --------')
print ('start time: {}'.format(start_time))
print ('end time: {}\n'.format(end_time))

duration = end_time-start_time
print ('training duration: {}'.format(duration))

torch.save(model.state_dict(), 'models/rainforest{}.pt'.format(end_time))

[1    50] loss: 3.153
[1   100] loss: 3.080
[1   150] loss: 3.093
[1   200] loss: 3.041
[1   250] loss: 2.996
[2    50] loss: 2.943
[2   100] loss: 2.866
[2   150] loss: 2.739
[2   200] loss: 2.649
[2   250] loss: 2.456
[3    50] loss: 2.364
[3   100] loss: 2.343
[3   150] loss: 2.204
[3   200] loss: 2.201
[3   250] loss: 2.214
[4    50] loss: 1.955
[4   100] loss: 1.992
[4   150] loss: 1.930
[4   200] loss: 1.768
[4   250] loss: 1.760
[5    50] loss: 1.754
[5   100] loss: 1.738
[5   150] loss: 1.598
[5   200] loss: 1.459
[5   250] loss: 1.417
[6    50] loss: 1.406
[6   100] loss: 1.346
[6   150] loss: 1.395
[6   200] loss: 1.450
[6   250] loss: 1.292
[7    50] loss: 1.162
[7   100] loss: 1.157
[7   150] loss: 1.159
[7   200] loss: 1.215
[7   250] loss: 1.039
[8    50] loss: 0.982
[8   100] loss: 1.225
[8   150] loss: 1.079
[8   200] loss: 1.044
[8   250] loss: 0.840
[9    50] loss: 0.804
[9   100] loss: 0.980
[9   150] loss: 0.825
[9   200] loss: 0.864
[9   250] loss: 0.848
[10    50]

In [7]:
# visualize

# for idx in range(len(train_classified)):
#     ex = train_classified.iloc[idx]
#     mfcc = ex.mfcc
#     fig, ax = plt.subplots()
#     img = librosa.display.specshow(mfcc, x_axis='time', ax=ax)
#     fig.colorbar(img, ax=ax)
#     ax.set(title='MFCC')

In [44]:
import wave
import struct

def signal_to_wav(signal, fname, Fs):
    """Convert a numpy array into a wav file.

     Args
     ----
     signal : 1-D numpy array
         An array containing the audio signal.
     fname : str
         Name of the audio file where the signal will be saved.
     Fs: int
        Sampling rate of the signal.

    """
    data = struct.pack('>' + ('d'*len(signal)), *signal)
    wav_file = wave.open(fname, 'wb')
    wav_file.setnchannels(1)
    wav_file.setsampwidth(2)
    wav_file.setframerate(Fs)
    wav_file.writeframes(data)
    wav_file.close()

In [7]:
# this will currently not work since the waveforms are being dropped in the dataframe
# wv, sample_rate = trainset.get_waveform(6)
# wv = wv.view(-1).numpy()

# plt.plot(wv);
# plt.title('Signal');
# plt.xlabel('Time (samples)');
# plt.ylabel('Amplitude');

In [10]:
test_files = glob.glob( 'data/test/*.flac' )

In [13]:
#test_files = pd.read_csv('test_files-20-percent.csv')

test_res = model = RainforestNet()
model.load_state_dict(torch.load('models/rainforest1608410308.1312122.pt'))
test_res = test_res.to(device)

In [14]:
#test_files.head()

In [35]:
torch.set_printoptions(sci_mode=False)
np.set_printoptions(suppress=True)

In [42]:
results = pd.DataFrame()

for f in tqdm(range(len(test_files))):
    #ex = test_files.iloc[idx]
    path = test_files[f]
    seconds = 3
    waveform, sample_rate = torchaudio.load(path)
    
    clips = waveform.split(sample_rate*seconds, dim=1)
    tot = torch.zeros(24).to(device)
    with torch.no_grad():
        for clip in clips:
            mel_spec = transforms.MelSpectrogram(sample_rate=sample_rate, n_mels=N_MELS)(clip)
            mel_spec = reshape_mel_spec(mel_spec)
            mel_spec = mel_spec.unsqueeze(0)
            mel_spec = mel_spec.to(device)
            
            output = test_res(mel_spec)
            #print (F.relu(output))
            
            tot = tot+output[0]
    
    tot = F.softmax(tot)
    tot = tot.to(torch.device('cpu')).numpy()
    
    results = results.append({ 'result': tot }, ignore_index=True)
    
    if f%10==0:
        print (f)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990



In [48]:
np.set_printoptions(suppress=True, threshold=sys.maxsize)
results.head()

,result
0,"[4.116101e-30, 0.0, 3.4954308e-25, 1e-45, 5.05..."
1,"[1.4901689e-33, 0.0, 1.4919007e-22, 7e-45, 9.8..."
2,"[1.7477177e-27, 0.0, 1.6326106e-27, 4.2e-44, 1..."
3,"[8.163591e-18, 1.15e-43, 1.8725104e-17, 1e-45,..."
4,"[1.108325e-27, 2e-44, 2.234145e-18, 4.921e-42,..."


In [58]:
submission = pd.DataFrame()

for idx in range(len(results)):
    ex = results.iloc[idx]
    recording_id = test_files[idx].replace('.flac', '').replace('data/test\\', '')
    result = ex.result
    
    species = 0
    o = {}
    for r in result:
        s = 's{}'.format(species)
        o[s] = r*100
        species += 1 
    o["recording_id"] = recording_id
    submission = submission.append(o, ignore_index=True)

In [60]:
cols = ['recording_id', 's0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 's22', 's23']
submission = submission[cols]

In [61]:
submission.head()

,recording_id,s0,s1,s2,s3,s4,s5,s6,s7,s8,...,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
0,000316da7,4.116101e-30,0.000000e+00,3.495431e-25,1.401298e-45,5.059540e-19,3.339823e-06,1.639266e-10,5.764734e-12,6.122568e-12,...,0.000000e+00,5.424553e-12,9.149371e-22,7.070026e-01,0.0,3.970402e-33,1.449799e-11,3.523845e-40,7.488090e-23,3.546282e-05
1,003bc2cb2,1.490169e-33,0.000000e+00,1.491901e-22,7.006492e-45,9.868375e-18,3.792711e-09,6.027892e-12,1.160006e-12,3.688508e-13,...,0.000000e+00,1.262131e-10,1.596878e-17,9.999995e-01,0.0,4.353403e-31,7.183413e-15,7.693129e-42,2.968396e-26,2.982222e-10
2,0061c037e,1.747718e-27,0.000000e+00,1.632611e-27,4.203895e-44,1.223990e-22,5.365190e-05,4.630734e-06,5.559545e-12,5.103002e-19,...,0.000000e+00,8.052121e-15,1.140547e-18,2.617697e-01,0.0,2.938265e-27,1.035853e-11,5.674138e-41,5.537738e-18,8.970207e-07
3,010eb14d3,8.163591e-18,1.149065e-43,1.872510e-17,1.401298e-45,3.879752e-03,9.603854e-01,3.421812e-09,1.641672e-09,1.893306e-04,...,1.914174e-41,1.368147e-11,1.343394e-17,5.700345e-10,0.0,3.049855e-28,3.892208e-18,9.606209e-29,3.924876e-14,4.517161e-04
4,011318064,1.108325e-27,1.961818e-44,2.234145e-18,4.921360e-42,4.219432e-13,5.183390e-06,2.475365e-07,2.927066e-10,1.750375e-11,...,1.401298e-45,1.802389e-09,7.492314e-16,9.999164e-01,0.0,1.398209e-26,2.118640e-12,1.197132e-36,1.313350e-19,4.641252e-07


In [64]:
np.set_printoptions(suppress=True, threshold=sys.maxsize)
submission.to_csv("submission-resnet-1.csv", index=False, float_format='%.5f')